### How many games does a team needs to win to be crowned champions?

First we will need to import the standing tables

In [1]:
import re
import numpy as np
import pandas as pd
from library import (
    get_squad_as_index,
    remove_unnamed_cols,
    clean_opp_df,
    clean_main_df,
    merge_dfs,
    get_single_season_league_data,
)


In [2]:
my_list_first = pd.read_html("https://fbref.com/en/comps/9/2021-2022/")
leagues_list = my_list_first.copy()

In [3]:
len(leagues_list)

24

In [4]:
shooting_dfs = merge_dfs(leagues_list[8], leagues_list[9])
shooting_dfs.head(5)

Standard                                                      ...  \
                 Gls   Sh  SoT  SoT%  Sh/90 SoT/90  G/Sh G/SoT  Dist  FK  ...   
Arsenal           60  581  186  32.0  15.29   4.89  0.09  0.30  17.8  22  ...   
Aston Villa       50  461  159  34.5  12.13   4.18  0.10  0.30  17.6  20  ...   
Brentford         46  436  141  32.3  11.47   3.71  0.09  0.28  16.3  11  ...   
Brighton          40  482  141  29.3  12.68   3.71  0.07  0.26  17.7  15  ...   
Burnley           32  405  119  29.4  10.66   3.13  0.08  0.26  17.2  14  ...   

                                                       Expected           \
            G/SoT_Opp Dist_Opp FK_Opp PK_Opp PKatt_Opp   xG_Opp npxG_Opp   
Arsenal          0.30     17.6   16.0    5.0       6.0     51.2     46.5   
Aston Villa      0.33     17.4   22.0    4.0       6.0     54.4     49.6   
Brentford        0.28     18.2   22.0    5.0       5.0     54.0     50.0   
Brighton         0.29     18.6   21.0    3.0       4.0     49.8     46.9   
Burnley          0.27     17.7   16.0    5.0       5.0     63.8     59.9   

                                              
            npxG/Sh_Opp G-xG_Opp np:G-xG_Opp  
Arsenal            0.11     -4.2        -4.5  
Aston Villa        0.11     -2.4        -1.6  
Brentford          0.10      0.0        -1.0  
Brighton           0.10     -5.8        -5.9  
Burnley            0.10    -12.8       -13.9  

[5 rows x 34 columns]

In [5]:
passing_df = merge_dfs(leagues_list[10], leagues_list[11])
passing_df.head(5)

Total                       Short             Medium              \
               Att  Cmp% TotDist PrgDist   Cmp   Att  Cmp%    Cmp   Att  Cmp%   
Arsenal      19844  81.5  280868   91115  7325  8210  89.2   7187  8140  88.3   
Aston Villa  16499  77.3  220973   85417  5916  6803  87.0   5184  6136  84.5   
Brentford    16272  71.8  211867   85979  5201  6312  82.4   4797  6120  78.4   
Brighton     20588  79.7  288827   95329  7195  8172  88.0   7130  8208  86.9   
Burnley      14107  67.0  178782   80196  4296  5327  80.6   3306  4472  73.9   

             ...     Long Details                                          \
             ... Cmp%_Opp Ast_Opp xAG_Opp xA_Opp A-xAG_Opp KP_Opp 1/3_Opp   
Arsenal      ...     50.4    33.0    37.3   27.7      -4.3  316.0   996.0   
Aston Villa  ...     58.0    35.0    37.7   30.1      -2.7  348.0  1094.0   
Brentford    ...     48.7    41.0    38.8   32.3       2.2  359.0  1331.0   
Brighton     ...     49.9    34.0    34.9   25.7      -0.9  342.0   996.0   
Burnley      ...     50.7    36.0    46.1   35.7     -10.1  459.0  1220.0   

                                        
            PPA_Opp CrsPA_Opp Prog_Opp  
Arsenal       255.0      71.0    838.0  
Aston Villa   276.0      61.0   1030.0  
Brentford     339.0     101.0   1030.0  
Brighton      255.0      58.0    886.0  
Burnley       407.0      83.0   1213.0  

[5 rows x 44 columns]

In [6]:
squad_pass_types_df = merge_dfs(leagues_list[12], leagues_list[13])
squad_pass_types_df.head()

Pass_Types                                    Corner_Kicks      \
                  Live  Dead   FK  TB   Sw  Crs   TI   CK           In Out   
Arsenal          18150  1628  404  77  132  633  679  208          153  23   
Aston Villa      14615  1832  525  71   97  670  716  196          111  42   
Brentford        14443  1763  432  50   70  640  778  159           84  47   
Brighton         18838  1697  390  58  152  728  784  200           84  96   
Burnley          12275  1731  370  32  135  747  783  183          152  14   

             ... Pass_Types                       Corner_Kicks          \
             ...     Sw_Opp Crs_Opp TI_Opp CK_Opp       In_Opp Out_Opp   
Arsenal      ...      102.0   606.0  744.0  175.0         90.0    50.0   
Aston Villa  ...      233.0   685.0  815.0  220.0        118.0    64.0   
Brentford    ...      104.0   812.0  774.0  229.0        128.0    69.0   
Brighton     ...      113.0   680.0  740.0  211.0        106.0    55.0   
Burnley      ...      168.0   861.0  846.0  265.0         95.0    79.0   

                    Outcomes                     
            Str_Opp  Cmp_Opp Off_Opp Blocks_Opp  
Arsenal         3.0  14226.0    64.0      295.0  
Aston Villa     3.0  15221.0    39.0      373.0  
Brentford       2.0  15770.0    71.0      329.0  
Brighton        6.0  13304.0    42.0      374.0  
Burnley         5.0  16820.0    54.0      364.0  

[5 rows x 28 columns]

In [7]:
goal_and_shot = merge_dfs(leagues_list[14], leagues_list[15])
goal_and_shot.head()

SCA        SCA_Types                           GCA        ...  \
              SCA  SCA90  PassLive PassDead Drib  Sh Fld Def GCA GCA90  ...   
Arsenal      1029  27.08       787       68   57  66  39  12  99  2.61  ...   
Aston Villa   836  22.00       609       84   48  41  48   6  93  2.45  ...   
Brentford     740  19.47       488       96   37  62  43  14  79  2.08  ...   
Brighton      868  22.84       631       84   47  54  39  13  72  1.89  ...   
Burnley       674  17.74       455       81   28  50  50  10  60  1.58  ...   

            SCA_Types         GCA_Opp              GCA_Types               \
              Fld_Opp Def_Opp GCA_Opp GCA90_Opp PassLive_Opp PassDead_Opp   
Arsenal          39.0    14.0    79.0      2.08         57.0          3.0   
Aston Villa      51.0    19.0    91.0      2.39         63.0          3.0   
Brentford        49.0    11.0    97.0      2.55         73.0          6.0   
Brighton         41.0    11.0    77.0      2.03         49.0         11.0   
Burnley          38.0    13.0    92.0      2.42         75.0          1.0   

                                             
            Drib_Opp Sh_Opp Fld_Opp Def_Opp  
Arsenal          6.0    5.0     7.0     1.0  
Aston Villa      6.0   10.0     8.0     1.0  
Brentford        2.0    8.0     7.0     1.0  
Brighton         4.0    7.0     4.0     2.0  
Burnley          2.0    8.0     6.0     0.0  

[5 rows x 32 columns]

In [8]:
squad_defensive_actions = merge_dfs(leagues_list[16], leagues_list[17])

# squad_defensive_actions.rename(columns={})
squad_defensive_actions.head()

Tackles                              Vs_Dribbles                  \
                Tkl TklW Def_3rd Mid_3rd Att_3rd         Tkl  Att  Tkl% Past   
Arsenal         540  311     261     195      84         244  500  48.8  256   
Aston Villa     634  382     300     259      75         300  649  46.2  349   
Brentford       607  342     299     228      80         291  616  47.2  325   
Brighton        668  403     306     257     105         243  563  43.2  320   
Burnley         590  324     273     235      82         250  534  46.8  284   

            Blocks  ... Vs_Dribbles                   Blocks_Opp         \
            Blocks  ...     Att_Opp Tkl%_Opp Past_Opp Blocks_Opp Sh_Opp   
Arsenal        386  ...       590.0     45.9    319.0      428.0  164.0   
Aston Villa    453  ...       587.0     43.3    333.0      356.0  127.0   
Brentford      383  ...       476.0     45.0    262.0      423.0  132.0   
Brighton       480  ...       559.0     43.3    317.0      481.0  140.0   
Burnley        509  ...       479.0     46.1    258.0      369.0  104.0   

                     Details                              
            Pass_Opp Int_Opp Tkl+Int_Opp Clr_Opp Err_Opp  
Arsenal        264.0   364.0       971.0   737.0    18.0  
Aston Villa    229.0   339.0       937.0   743.0     6.0  
Brentford      291.0   381.0       886.0   830.0    12.0  
Brighton       341.0   389.0       980.0   752.0     8.0  
Burnley        265.0   281.0       793.0   853.0     9.0  

[5 rows x 32 columns]

In [9]:
squad_possession = merge_dfs(leagues_list[18], leagues_list[19])
squad_possession.head()

Touches                                                Dribbles  \
            Touches Def_Pen Def_3rd Mid_3rd Att_3rd Att_Pen   Live     Succ   
Arsenal       23628    2229    7136   10298    6399     962  23621      319   
Aston Villa   20477    2409    6868    8846    4984     814  20474      333   
Brentford     20340    2790    7848    8156    4515     807  20334      262   
Brighton      24676    2636    8026   10462    6403     861  24669      317   
Burnley       18254    2240    5824    8007    4637     659  18252      258   

                        ... Touches_Opp                      Dribbles          \
             Att Succ%  ... Att_3rd_Opp Att_Pen_Opp Live_Opp Succ_Opp Att_Opp   
Arsenal      590  54.1  ...      4999.0       712.0  21807.0    256.0   500.0   
Aston Villa  587  56.7  ...      5311.0       843.0  22909.0    349.0   649.0   
Brentford    476  55.0  ...      6689.0       828.0  24279.0    325.0   616.0   
Brighton     559  56.7  ...      5286.0       817.0  21426.0    320.0   564.0   
Burnley      479  53.9  ...      6535.0      1077.0  25474.0    284.0   534.0   

                                      Receiving           
            Succ%_Opp Mis_Opp Dis_Opp   Rec_Opp Prog_Opp  
Arsenal          51.2   550.0   296.0   14022.0    824.0  
Aston Villa      53.8   541.0   334.0   15022.0   1011.0  
Brentford        52.8   618.0   316.0   15560.0   1011.0  
Brighton         56.7   581.0   425.0   13089.0    860.0  
Burnley          53.2   571.0   340.0   16592.0   1187.0  

[5 rows x 28 columns]

In [11]:
squad_possession.columns

MultiIndex([(    'Touches',     'Touches'),
            (    'Touches',     'Def_Pen'),
            (    'Touches',     'Def_3rd'),
            (    'Touches',     'Mid_3rd'),
            (    'Touches',     'Att_3rd'),
            (    'Touches',     'Att_Pen'),
            (    'Touches',        'Live'),
            (   'Dribbles',        'Succ'),
            (   'Dribbles',         'Att'),
            (   'Dribbles',       'Succ%'),
            (   'Dribbles',         'Mis'),
            (   'Dribbles',         'Dis'),
            (  'Receiving',         'Rec'),
            (  'Receiving',        'Prog'),
            ('Touches_Opp', 'Touches_Opp'),
            ('Touches_Opp', 'Def_Pen_Opp'),
            ('Touches_Opp', 'Def_3rd_Opp'),
            ('Touches_Opp', 'Mid_3rd_Opp'),
            ('Touches_Opp', 'Att_3rd_Opp'),
            ('Touches_Opp', 'Att_Pen_Opp'),
            ('Touches_Opp',    'Live_Opp'),
            (   'Dribbles',    'Succ_Opp'),
            (   'Dribbles',     

In [ ]:
"""
Squad Possession
- Touches -->
    :Touches: Number of touches on the ball
    :Def_Pen: Touches in Defending Penalty Area
    :Def_3rd: Touches in defensive 3rd
    :Mid_3rd: Touches in middle 3rd
    :Att_3rd: Touches in attacking 3rd
    :Att_Pen: Touches in Attacking penalty area
    :Live: Live Ball Touches
- Dribbles -->
    :Succ: Number of successful dribbles
    :Att: Number of attempted dribbles
    :Succ%: Percentage of successful dribbles
    :Mis: Number of times a player failed to gain control of the ball
    :Dis: Number of times a player lost control of the ball after been
          tackled
- Receiving -->
    :Rec: Number of times a player successfully received a pass
    :Prog: Progressive Passes Received: Completed passes that move the
           ball towards the opponent's goal at least 10 yards from its
           furthest point in the last six passes, or any completed pass
           into the penalty area. 
           Excludes passes from the defending 40% of the pitch
"""